In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
# URL
url = "https://u.gg/lol/profile/euw1/leaguify/champion-stats?queueType=normal_draft_5x5"

options = Options()
options.add_argument("--headless")
options.binary_location = "C:\\Users\\nicol\\Downloads\\chrome-win64\\chrome-win64\\chrome.exe"

service = Service("C:\\Users\\nicol\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.rt-tr-group")))

# Initialize list to store each row of data
data = []

# Get page source and create BeautifulSoup object
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close the driver
driver.quit()

# Get all the rows using BeautifulSoup
rows = soup.select("div.rt-tr-group")

for i, row in enumerate(rows, 1):  # start from 1 because CSS nth-child starts from 1
    try:
        selectors = {
            'Champion': "div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(2)",
            'WinsLoses': "div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > span:nth-child(3)",
            'Winrate': "div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > strong:nth-child(1)",
            'KDA': "div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > strong:nth-child(1)",
            'KillsDeathsAssists': "div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > span:nth-child(2)",
            'LP': "div:nth-child(1) > div:nth-child(5) > span:nth-child(1)",
            'MaxKills': "div:nth-child(1) > div:nth-child(6) > span:nth-child(1)",
            'MaxDeaths': "div:nth-child(1) > div:nth-child(7)",
            'CS': "div:nth-child(1) > div:nth-child(8) > span:nth-child(1)",
            'Damage': "div:nth-child(1) > div:nth-child(9) > span:nth-child(1)",
            'Gold': "div:nth-child(1) > div:nth-child(10) > span:nth-child(1)"
        }

        row_data = []
        for key, selector in selectors.items():
            element = row.select_one(selector)
            text = element.get_text(strip=True) if element else 'N/A'
            row_data.append(text)

        data.append(row_data)

    except Exception as e:
        print(f"Error in row {i}: {e}")

# Convert the data to a DataFrame
columns = ['Champion', 'WinsLoses', 'Winrate', 'KDA', 'KillsDeathsAssists', 'LP', 'MaxKills', 'MaxDeaths', 'CS', 'Damage', 'Gold']
df_individual = pd.DataFrame(data, columns=columns)

2023-09-23 14:32:18 [scrapy.addons] INFO: Enabled addons:
[]
2023-09-23 14:32:18 [py.warnings] WARNING: C:\Programmieren\Leaguify_Website\venv\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)

2023-09-23 14:32:18 [scrapy.extensions.telnet] INFO: Telnet Password: 11dbb76e787f83ce
2023-09-23 14:32:18 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2023-0

CSV data:
Rank,Champion,Win Rate,Wins/Loses,Unnamed,Kills,Deaths,Assists,LP,Max Kills,Max Deaths,CS,Damage,Gold
1,Rell,56%,69,2.65,1.8,6.9,16.6,165,7,15,28.9,"6,903","7,644"
2,Alistar,57%,66,2.36,2.1,6.9,14.3,234,7,14,28.6,"8,333","7,576"
3,Leona,52%,58,2.38,1.9,6.8,14.3,169,7,16,32.4,"7,364","7,640"
4,Thresh,49%,17,2.31,1.6,6.9,14.3,5,4,14,32.6,"7,227","7,394"
5,Rakan,44%,10,2.16,1.4,6.5,12.7,110,4,15,32.3,"5,830","6,710"
6,Blitzcrank,40%,4,1.58,1.1,7.7,11.1,55,3,16,24.7,"6,804","6,087"
7,Pyke,38%,3,1.21,5.4,10.8,7.6,8,7,16,34.4,"7,724","8,646"
8,Karma,33%,2,2.10,2.3,6.8,12.0,26,6,9,19.5,"10,798","7,360"
9,Braum,50%,2,1.84,1.5,9.3,15.5,21,4,12,31.8,"8,860","7,785"
10,Akshan,50%,1,1.71,7.0,7.0,5.0,26,11,10,186.5,"19,388","11,494"
11,Janna,50%,1,6.00,0.5,3.0,17.5,N/A,1,5,10.5,"4,111","6,290"
12,Lissandra,0%,0,1.89,0.5,9.0,16.5,51,1,14,16.5,"10,017","7,541"
13,Nautilus,0%,0,1.21,2.0,9.5,9.5,52,3,10,30.5,"5,978","5,958"
14,Veigar,0%,0,1.11,4.5,9.0,5.5,51,7,12,176,"18,102","10,405"
15,Xaya

In [ ]:
df_individual